In [1]:
import pandas as pd
import requests
from datetime import datetime
from bs4 import BeautifulSoup
from utils import read_config, MySQLAgent
import time, random

In [2]:
config = read_config('.env/local_conn.json')
sql_agent = MySQLAgent(config['local_mysql'])

In [4]:
query = """
show tables
"""

sql_agent.read_table(query=query)

,tables_in_generaldb
0,balancesheet
1,port_activity


In [53]:
query = """
select *
from balancesheet
where stock = '2330'
"""

df_balance = sql_agent.read_table(query=query)

In [55]:
df_balance['period_time'] = df_balance['period_year'].astype(str) + '-' + df_balance['season'].astype(str)

In [56]:
total_assests = df_balance[(df_balance['acct_name'] == '資產總額') & (df_balance['season'] == 4)][['period_time', 'this_year_amt']].reset_index(drop=True).rename(columns={'this_year_amt':'total_assests_by_season'})

In [57]:
total_assests['total_assests_varation'] = total_assests['total_assests_by_season'].str.replace(',', '').astype(int).pct_change()*100
total_assests

,period_time,total_assests_by_season,total_assests_varation
0,108-4,"2,264,805,032",NaN
1,109-4,"2,760,711,405",21.896206
2,110-4,"3,725,503,455",34.947226
3,111-4,"4,964,778,878",33.264643
4,112-4,"5,532,371,215",11.432379
